In [1]:
!pip install --quiet python-magic langchain llama_index unstructured openai chromadb pdfminer.six pytesseract tiktoken deeplake selenium faiss-cpu

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\http\client.py", line 464, in read
    s = self.fp.read(amt)
  File "C:\Users\Tanveer\anaconda3\envs\halalbot\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Tanveer\anaconda3\envs\

In [ ]:
#!pip install gradio

In [3]:
import pandas as pd
import nltk
import os

import langchain
from langchain.schema import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.document_loaders import OnlinePDFLoader
from langchain.llms import OpenAIChat

from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain'

In [2]:
# Set your OpenAI API key
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")
# Set your OpenAI API key
openai_api_key = os.environ['OPENAI_API_KEY']


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key=os.getenv("openai_key")

In [7]:
import csv

def read_protocol_reports(file_path):
    protocol_reports = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        protocol_name = None
        report = ""
        for line in lines:
            if line.strip().startswith("Name of the Protocol:"):
                if protocol_name is not None:
                    protocol_reports.append((protocol_name.strip(), report.strip()))
                protocol_name = line[len("Name of the Protocol:"):].strip()
                report = ""
            else:
                report += line
        # Append the last protocol report
        if protocol_name is not None:
            protocol_reports.append((protocol_name.strip(), report.strip()))
    return protocol_reports

def write_to_csv(protocol_reports, csv_file):
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['protocol_name', 'report'])
        for protocol, report in protocol_reports:
            csvwriter.writerow([protocol, report])

if __name__ == "__main__":
    file_path = "report.txt"  # Path to your text file
    csv_file = "protocol_reports.csv"    # Desired name for the CSV file
    protocol_reports = read_protocol_reports(file_path)
    write_to_csv(protocol_reports, csv_file)


# Report ChatBot

In [12]:
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_key)

C:\Users\Bismillah Traders\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [13]:
# load  vector store
reportsearch = FAISS.load_local("faiss_index_report", embeddings)

In [14]:
template = """Please provide accurate and intelligent responses to our clients as a virtual assistant specializing in Crypto Currencies. Begin the conversation with a friendly greeting. When discussing information or providing recommendations about 
Crypto Currencies, make sure to use accurate information from the knowledge base provided in {context}.If a question is asked that 
is not related to Crypto Currencies or falls outside the scope of this document, kindly reply with the response, \"I'm sorry, 
but the available information is limited as I am an AI assistant.\" Please refer to the chat history in {chat_history} and 
respond to the human input as follows: \"Human: {human_input} Virtual Assistant:\""""




prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory_report = ConversationBufferMemory(memory_key="chat_history", input_key="human_input", max_history=2)
chain_report = load_qa_chain(ChatOpenAI(model_name="gpt-4", temperature=0,max_tokens=2000), chain_type="stuff", memory=memory_report, prompt=prompt)


C:\Users\Bismillah Traders\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [35]:
query = "is smardex a good investment?"

In [36]:
docs = reportsearch.similarity_search(query)

In [37]:
docs

[Document(page_content="Potential Shariah Concerns in SmarDex Protocol and SDX Token:\n* Gambling (قمار, ميسر): There's no indication that SDX is particularly used for gambling or speculative activities.\n* Interest (Usury): The tokens or the system does not seem to be dependent on interest-based transactions. However, staking rewards or any other returns need to be separately evaluated. If the farming and staking are not fixed and interest-based, they are permissible.\n23.4. Shariah Opinion of the SmarDex Protocol and SDX Token:\nIn conclusion, from a Shariah perspective, both the SmarDex Protocol and the SDX token can be characterized as permissible. However, it does not mean that all the listed tokens and the products on the platform are also Shariah compliant. The permissibility of the products is determined by their application and usage. Users should also ensure that their interactions/transactions, such as trading, investment, and holding, with the protocol or token are aligned 

In [38]:
output = chain_report({"input_documents": [docs[0]], "human_input": query}, return_only_outputs=False)

In [39]:
output['output_text']

"As an AI assistant, I'm not able to provide personal investment advice. However, I can tell you that the SmarDex Protocol and SDX token are considered permissible from a Shariah perspective. They are not used for gambling or speculative activities, and the system is not dependent on interest-based transactions. However, this does not mean that all tokens and products on the platform are also Shariah compliant. The permissibility is determined by their application and usage. It's always recommended to do your own research and consider your own financial situation and risk tolerance before making any investment decisions."

In [ ]:
# Langchain url loader

In [ ]:
from googleapiclient.discovery import build
import pprint

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

if __name__ == '__main__':
    api_key = 'YOUR_API_KEY'
    cse_id = 'YOUR_CSE_ID'

    search_term = 'btc'
    results = google_search(search_term, api_key, cse_id, num=3)

    # Extract and print the top 3 URLs
    top_links = [item['link'] for item in results['items']]
    pprint.pprint(top_links)
